In [39]:
import pandas as pd
import datetime

%matplotlib inline
#userLog = pd.read_csv("./clean_data/userlog_feature.csv")

In [27]:
userLog['projects_per_hour'] = userLog['user_clicked_by_hour']/userLog['user_clicked_projects_by_hour']
userLog = userLog[userLog['projects_per_hour']<100]
print(userLog.shape)

(1232752, 13)


/home/users/u5722792744_1/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
# userLog['datetime'] = userLog.apply(lambda row : datetime.datetime(row['year'], row['month'], row['day'], row['hour']), axis=1)
# userLog['date'] = userLog['datetime'].map(lambda x : x.date())
# userLog['yearmonth'] = userLog['date'].map(lambda x: str(x.year) +'-'+ str(x.month).zfill(2))
# userLog.head(3)

In [32]:
# # Filter out user that is a bot

project_count = userLog.groupby(['userCode']).size()
select_project = set(project_count[project_count < 1000].index)

userLog2 = userLog[userLog['userCode'].isin(select_project)]
print(userLog2.shape)
userLog2.head()

(1194706, 13)


userCode  project_id requestedDevice userAgent  \
0  7717bdc2-ea3e-e8ad-5d6b-178bd71c38b2        7956          Mobile   Android   
1  7717bdc2-ea3e-e8ad-5d6b-178bd71c38b2        7956          Mobile   Android   
2  cb5b4b68-cc01-6db6-f54b-4a0f881301c5        5067          Mobile    iPhone   
3  5f74cef2-0d1e-b619-3564-0955a14e0985        6654          Mobile    iPhone   
4  dba8f279-844e-eef6-73ac-22bd7d1353cc        6474          Mobile      iPad   

  pageReferrer  year  month  day  hour             datetime  \
0  HomeWebsite  2018      1    1     0  2018-01-01 00:00:00   
1  HomeWebsite  2018      1    1     0  2018-01-01 00:00:00   
2  HomeWebsite  2018      1    1     0  2018-01-01 00:00:00   
3       Google  2018      1    1     0  2018-01-01 00:00:00   
4       Google  2018      1    1     0  2018-01-01 00:00:00   

   user_clicked_by_hour  user_clicked_projects_by_hour  projects_per_hour  
0                     3                              3                1.0  
1                     3                              3                1.0  
2                     1                              1                1.0  
3                     1                              1                1.0  
4                     2                              2                1.0

In [ ]:
# min_interacted = 30
# project_count = userLog.groupby(['project_id']).size()
# ignore_project = set(project_count[project_count > min_interacted].index)
# print(len(ignore_project))

# df_filter = userLog[~userLog['project_id'].isin(ignore_project)]
# df_filter.head(3)

In [33]:
userLog2['interacted'] = 1

/home/users/u5722792744_1/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [34]:
# Convert the dataframe into a matrix using pivot() and set index of df_train for fast search 
df_train_indexed = userLog2[['userCode', 'project_id', 'interacted']].drop_duplicates()
df_train_pivot = (df_train_indexed.pivot(index = 'userCode', columns = 'project_id', values = 'interacted')
                                  .fillna(0))
df_train_pivot.head(3) 

project_id                            4     24    29    41    44    45    \
userCode                                                                   
00005aba-5ebc-0821-f5a9-bacca40be125   0.0   0.0   0.0   0.0   0.0   0.0   
0000bae7-6233-d7cc-2a6d-48aa70fe8ad4   0.0   0.0   0.0   0.0   0.0   0.0   
0000c576-e929-19eb-615a-349ec3b4709b   0.0   0.0   0.0   0.0   0.0   0.0   

project_id                            73    95    115   123   ...   9489  \
userCode                                                      ...          
00005aba-5ebc-0821-f5a9-bacca40be125   0.0   0.0   0.0   0.0  ...    0.0   
0000bae7-6233-d7cc-2a6d-48aa70fe8ad4   0.0   0.0   0.0   0.0  ...    0.0   
0000c576-e929-19eb-615a-349ec3b4709b   0.0   0.0   0.0   0.0  ...    0.0   

project_id                            9491  9492  9493  9494  9496  9499  \
userCode                                                                   
00005aba-5ebc-0821-f5a9-bacca40be125   0.0   0.0   0.0   0.0   0.0   0.0   
0000bae7-6233-d7cc-2a6d-48aa70fe8ad4   0.0   0.0   0.0   0.0   0.0   0.0   
0000c576-e929-19eb-615a-349ec3b4709b   0.0   0.0   0.0   0.0   0.0   0.0   

project_id                            9500  9502  9504  
userCode                                                
00005aba-5ebc-0821-f5a9-bacca40be125   0.0   0.0   0.0  
0000bae7-6233-d7cc-2a6d-48aa70fe8ad4   0.0   0.0   0.0  
0000c576-e929-19eb-615a-349ec3b4709b   0.0   0.0   0.0  

[3 rows x 5290 columns]

In [35]:
import numpy as np
R = df_train_pivot.as_matrix()
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)


/home/users/u5722792744_1/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [36]:
R.shape
user_ratings_mean.reshape(-1, 1).shape

(482744, 1)

In [57]:
%%time
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned, k = 500)

CPU times: user 7h 27min 36s, sys: 19min 48s, total: 7h 47min 24s
Wall time: 29min 46s


In [65]:
import numpy as np
np.save(file("./var/U_k500.np"), U)
np.save(file("./var/sigma_k500.np"), sigma)
np.save(file("./var/Vt_k500.np"), Vt)

NameError: name 'file' is not defined

In [58]:
sigma = np.diag(sigma)

In [59]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = df_train_pivot.columns)

In [60]:
#preds_df = pd.DataFrame(preds_df, columns = df_train_pivot.columns)
preds_df['userCode'] = df_train_pivot.index
preds_df2 = preds_df.reset_index()
#del preds_df['project_id']

In [68]:
projs_id = preds_df.columns
def recommend_project(userId, preds_df=preds_df):
    score = preds_df.query("userCode=='%s'"%userId)
    seen_projs = list(userLog[userLog['userCode']=="ffffaaf1-e1ff-15bb-2b8f-5d757ea42825"]['project_id'].values)
    if score.shape[0]==0: return "5858 3738 4201 5452 5678 5764 5067"
    
    cal_df = pd.DataFrame({"proj_id":preds_df.columns,"score":score.values[0]})
    recommending = cal_df[cal_df['proj_id']!="userCode"].sort_values("score",ascending=False).head(7+len(seen_projs))

    ans = [a for a in recommending['proj_id'] if a not in seen_projs]
    return " ".join([str(i) for i in ans[:7]])
    
recommend_project("00019a11-0aaa-2559-4feb-534C24830d3e")

'5858 3738 4201 5452 5678 5764 5067'

### Predicting users

NameError: name 'score' is not defined

In [ ]:
%%time
def predictFromFile(file,submission_name,sep=','):    
    users = pd.read_csv(file,sep=sep)
    print(users.head())
    users['project_id'] = users['userCode'].apply(recommend_project)
    users.to_csv("./submission/%s.csv"%submission_name,index=False)
    
predictFromFile("./clean_data/test_before_021718_7000row.csv","matrix_factorise_v4_k500_ontest7000")

                               userCode            ans_project_id
0  5ea1f668-f96d-9544-38b6-d4c2e39f668c  [6042, 7026, 8091, 6991]
1  8065faa5-cfa4-e0c8-5ec5-22146af8665c                    [8955]
2  b87bb27a-7679-4631-ef12-075fa9ee1b6a                    [6221]
3  f2aad9cb-814b-bb9c-bce9-85b979d81588        [9066, 8412, 7694]
4  7cdef1ff-5dd9-8eb9-0fd2-28244bac1e87                    [6799]


In [63]:
%%time
predictFromFile("./data/testing_users.csv","matrix_factorise_v4_submission")

                               userCode
0  003366c6-0cd9-48e7-d134-2051d7360c2d
1  003b2f18-60b4-37be-c63d-0caead1afa97
2  003d571e-9de7-6de3-2cb6-2ae190cc7beb
3  00426ede-6813-949b-de2e-6d8b228dff1d
4  0043a9c7-cace-1aa1-3188-4b173bf28991
CPU times: user 5h 52min 45s, sys: 23min 37s, total: 6h 16min 23s
Wall time: 28min 24s


In [ ]:
%%time
users['project_id'] = users['userCode'].apply(recommend_project)

In [ ]:
users.to_csv("./submission/submission_v4_full.csv",index=False)

In [ ]:
users.head()

In [ ]:
users.shape